In [1]:
#Import our saved data from data_clean.ipynb
%store -r ev_sales_nation_df
%store -r electric_vehicle_df
%store -r charging_stations_df
%store -r income_df

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge, Lasso

In [3]:
#Setting up our data for merging
category_counts = electric_vehicle_df['County'].value_counts()
df1 = category_counts.reset_index()
df1.columns = ['County', 'Count']

print(df1)

          County  Count
0           King  75383
1      Snohomish  16429
2         Pierce  11017
3          Clark   8451
4       Thurston   5097
5         Kitsap   4725
6        Spokane   3542
7        Whatcom   3529
8         Benton   1756
9         Skagit   1599
10        Island   1593
11       Clallam    883
12     Jefferson    848
13        Chelan    844
14      San Juan    821
15        Yakima    782
16       Cowlitz    717
17         Mason    688
18         Lewis    554
19  Grays Harbor    513
20      Kittitas    484
21      Franklin    469
22         Grant    435
23   Walla Walla    350
24       Douglas    285
25       Whitman    249
26     Klickitat    234
27      Okanogan    198
28       Stevens    190
29       Pacific    170
30      Skamania    162
31        Asotin     59
32     Wahkiakum     47
33  Pend Oreille     45
34         Adams     40
35       Lincoln     37
36         Ferry     25
37      Columbia     16
38      Garfield      3


In [4]:
df1.plot(kind='bar')
plt.xlabel('County')
plt.ylabel('Count')
plt.show()

In [5]:
df2 = income_df.T.reset_index()

In [6]:
df2.columns.values[0] = 'County'

In [7]:
df2

,County,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Adams,24604.0,28024.0,27301.0,28328.0,31277.0,29537.0,29604.0,31806.0,31795.0,34073.0,33888.0,35292.0,37839.0,38306.0,38934.0,39235.0,39105.0,41298.0,42299.0,42455.0,41102.0,40656.0,41068.0,42354.0,43541.0,45712.0,47646.0,49501.0,48849.0,52870.0,53535.0,56421.0,57405.0,58406.0
1,Asotin,22897.0,24206.0,25110.0,26251.0,27359.0,27873.0,28288.0,29813.0,31499.0,32477.0,33524.0,32590.0,35920.0,37013.0,37080.0,37140.0,37657.0,39768.0,40558.0,42945.0,40035.0,39820.0,40171.0,41703.0,42174.0,43368.0,46107.0,47020.0,51767.0,50746.0,54776.0,53377.0,56438.0,59675.0
2,Benton,32593.0,35012.0,37093.0,39228.0,41153.0,42790.0,42540.0,42817.0,44057.0,45665.0,47044.0,49389.0,52598.0,53736.0,55409.0,57613.0,56200.0,54942.0,55361.0,57114.0,58496.0,60070.0,60608.0,62739.0,63710.0,63157.0,62071.0,62282.0,63502.0,67912.0,72847.0,75882.0,77339.0,78824.0
3,Chelan,24312.0,25833.0,27592.0,28746.0,30148.0,31547.0,32164.0,33918.0,35662.0,37175.0,37316.0,39439.0,41653.0,41731.0,42918.0,43696.0,44422.0,46522.0,44964.0,44013.0,46780.0,45478.0,46275.0,47265.0,51713.0,50825.0,53068.0,55109.0,60791.0,60747.0,59838.0,61546.0,64700.0,68016.0
4,Clallam,25434.0,27329.0,27861.0,28578.0,29340.0,29951.0,31163.0,32559.0,34770.0,36012.0,36449.0,30866.0,32939.0,33229.0,34020.0,34172.0,35050.0,37770.0,37926.0,40912.0,38647.0,38397.0,38886.0,41887.0,44824.0,45454.0,46241.0,48187.0,47767.0,55664.0,57571.0,54712.0,62623.0,65652.0
5,Clark,31800.0,33734.0,34250.0,35847.0,37418.0,38828.0,40525.0,43044.0,45705.0,47252.0,48376.0,49320.0,51610.0,50518.0,50520.0,50474.0,52108.0,56994.0,57621.0,57999.0,54370.0,54581.0,54951.0,56054.0,57852.0,61711.0,63639.0,66782.0,71922.0,71659.0,80407.0,76929.0,83790.0,87397.0
6,Columbia,22418.0,24478.0,24123.0,26136.0,28123.0,26779.0,29861.0,33041.0,30820.0,33060.0,33500.0,37360.0,38376.0,37328.0,37593.0,37341.0,36028.0,37211.0,41531.0,43508.0,39978.0,38474.0,38916.0,39461.0,39931.0,42886.0,39807.0,41517.0,43622.0,46550.0,56338.0,58220.0,59650.0,61115.0
7,Cowlitz,27866.0,29746.0,31038.0,31195.0,31957.0,33309.0,34191.0,35516.0,36738.0,38437.0,39797.0,35246.0,37266.0,37040.0,36516.0,36355.0,37958.0,39777.0,39954.0,41412.0,40572.0,40867.0,41406.0,42752.0,43409.0,39635.0,49997.0,48208.0,47132.0,54588.0,57316.0,64357.0,70104.0,76364.0
8,Douglas,27054.0,28708.0,29407.0,30932.0,31855.0,32688.0,32930.0,35420.0,36855.0,37733.0,38464.0,39789.0,40699.0,41062.0,41809.0,42524.0,43189.0,45383.0,45748.0,48325.0,46269.0,46159.0,46723.0,48050.0,49030.0,50878.0,50886.0,52364.0,54581.0,65913.0,63086.0,64768.0,64932.0,65096.0
9,Ferry,25170.0,27355.0,27236.0,27841.0,27829.0,29587.0,28910.0,30091.0,30489.0,30412.0,30388.0,31175.0,33129.0,33488.0,33182.0,33867.0,34452.0,34828.0,37001.0,38093.0,38284.0,36712.0,36921.0,37548.0,39596.0,41343.0,40340.0,42330.0,45546.0,42212.0,47463.0,47722.0,50565.0,53577.0


In [8]:
df3 = pd.merge(df1, df2, on ='County')
df3

,County,Count,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,King,75383,36179.0,38633.0,39831.0,41647.0,42234.0,43810.0,45397.0,46726.0,48271.0,51266.0,53157.0,53937.0,55220.0,56098.0,56952.0,61565.0,61225.0,63745.0,65489.0,67027.0,65877.0,65383.0,66294.0,68313.0,71122.0,75045.0,80998.0,84897.0,88466.0,93707.0,102338.0,102620.0,110351.0,118664.0
1,Snohomish,16429,36847.0,38820.0,39868.0,41670.0,42418.0,43741.0,44994.0,47416.0,50680.0,52342.0,53060.0,50870.0,52935.0,53174.0,54563.0,56736.0,58353.0,60975.0,63682.0,64289.0,63297.0,62034.0,62687.0,64033.0,64391.0,68637.0,75292.0,77985.0,81779.0,86435.0,89119.0,93589.0,99650.0,106104.0
2,Pierce,11017,30412.0,33380.0,33976.0,35417.0,36210.0,37105.0,38222.0,39878.0,42596.0,44333.0,45204.0,42555.0,44965.0,45581.0,47084.0,49151.0,50678.0,55506.0,56426.0,57674.0,56555.0,55531.0,56114.0,57162.0,57238.0,59998.0,59566.0,61042.0,65517.0,71208.0,78779.0,80236.0,85492.0,91092.0
3,Clark,8451,31800.0,33734.0,34250.0,35847.0,37418.0,38828.0,40525.0,43044.0,45705.0,47252.0,48376.0,49320.0,51610.0,50518.0,50520.0,50474.0,52108.0,56994.0,57621.0,57999.0,54370.0,54581.0,54951.0,56054.0,57852.0,61711.0,63639.0,66782.0,71922.0,71659.0,80407.0,76929.0,83790.0,87397.0
4,Thurston,5097,30976.0,33139.0,34851.0,36676.0,37766.0,38939.0,39971.0,41497.0,43748.0,45843.0,46975.0,48457.0,50885.0,51111.0,51243.0,52043.0,54914.0,57985.0,60576.0,63009.0,60978.0,60038.0,60621.0,62009.0,63408.0,65288.0,61676.0,63286.0,68765.0,69757.0,77890.0,79769.0,81544.0,83358.0
5,Kitsap,4725,32043.0,36145.0,37380.0,38404.0,39391.0,40407.0,40630.0,41740.0,44098.0,45667.0,46840.0,48387.0,50751.0,52051.0,52192.0,53227.0,57034.0,51610.0,53680.0,55417.0,56863.0,54804.0,55400.0,57155.0,60200.0,59605.0,65156.0,66569.0,70531.0,74150.0,79268.0,76814.0,86826.0,91737.0
6,Spokane,3542,25769.0,27337.0,28355.0,29570.0,30482.0,31631.0,32256.0,33517.0,34920.0,36389.0,37308.0,39401.0,40525.0,40872.0,42533.0,44836.0,44538.0,45753.0,47848.0,48876.0,46983.0,46320.0,46846.0,48265.0,48312.0,50856.0,48189.0,49482.0,49954.0,55535.0,59976.0,60827.0,65571.0,70685.0
7,Whatcom,3529,28367.0,31097.0,31953.0,32599.0,32958.0,34031.0,34893.0,36492.0,37553.0,39188.0,40005.0,37044.0,39301.0,39568.0,40486.0,41151.0,43372.0,46879.0,49778.0,50443.0,49761.0,49294.0,49775.0,51268.0,50879.0,53481.0,54522.0,55710.0,60028.0,61244.0,68656.0,70463.0,71739.0,73038.0
8,Benton,1756,32593.0,35012.0,37093.0,39228.0,41153.0,42790.0,42540.0,42817.0,44057.0,45665.0,47044.0,49389.0,52598.0,53736.0,55409.0,57613.0,56200.0,54942.0,55361.0,57114.0,58496.0,60070.0,60608.0,62739.0,63710.0,63157.0,62071.0,62282.0,63502.0,67912.0,72847.0,75882.0,77339.0,78824.0
9,Skagit,1599,28389.0,30181.0,30979.0,31970.0,32843.0,34089.0,35278.0,36888.0,38449.0,40582.0,42381.0,42972.0,45287.0,45747.0,46747.0,48229.0,49196.0,52104.0,53874.0,54803.0,55572.0,54426.0,55085.0,56443.0,56058.0,54852.0,56322.0,55524.0,60175.0,65182.0,67551.0,78798.0,72831.0,67316.0


In [9]:
df3.sort_values(by=['2022'])

,County,Count,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
33,Pend Oreille,45,20808.0,22660.0,23214.0,24690.0,26103.0,27123.0,28328.0,29785.0,31223.0,31684.0,31677.0,33513.0,34855.0,34208.0,35054.0,35143.0,35343.0,36737.0,37268.0,37680.0,37467.0,37005.0,37234.0,37755.0,42043.0,39886.0,41111.0,42391.0,42276.0,43299.0,51950.0,57212.0,55060.0,52989.0
36,Ferry,25,25170.0,27355.0,27236.0,27841.0,27829.0,29587.0,28910.0,30091.0,30489.0,30412.0,30388.0,31175.0,33129.0,33488.0,33182.0,33867.0,34452.0,34828.0,37001.0,38093.0,38284.0,36712.0,36921.0,37548.0,39596.0,41343.0,40340.0,42330.0,45546.0,42212.0,47463.0,47722.0,50565.0,53577.0
25,Whitman,249,21674.0,22949.0,23735.0,24389.0,25162.0,25134.0,26270.0,27838.0,28697.0,29174.0,28584.0,24596.0,24841.0,24805.0,25869.0,26752.0,26788.0,28303.0,31302.0,32604.0,32037.0,31062.0,31396.0,32570.0,43091.0,42218.0,43379.0,49946.0,52543.0,53100.0,45906.0,52066.0,53140.0,54236.0
19,Grays Harbor,513,23042.0,24053.0,24677.0,26038.0,27002.0,27252.0,28638.0,29792.0,31368.0,33167.0,34160.0,36410.0,37440.0,38102.0,38305.0,38367.0,39943.0,42029.0,43126.0,42646.0,39927.0,39452.0,39836.0,40354.0,40323.0,41687.0,43902.0,44627.0,44080.0,45139.0,59346.0,54034.0,55018.0,56020.0
27,Okanogan,198,20303.0,20580.0,22203.0,23697.0,25074.0,25999.0,25495.0,27576.0,28047.0,29598.0,29726.0,28659.0,30137.0,31451.0,32725.0,34497.0,34659.0,36798.0,35712.0,36681.0,35227.0,34915.0,35161.0,35848.0,36292.0,35146.0,41426.0,41028.0,44069.0,47536.0,44777.0,48979.0,55233.0,57422.0
34,Adams,40,24604.0,28024.0,27301.0,28328.0,31277.0,29537.0,29604.0,31806.0,31795.0,34073.0,33888.0,35292.0,37839.0,38306.0,38934.0,39235.0,39105.0,41298.0,42299.0,42455.0,41102.0,40656.0,41068.0,42354.0,43541.0,45712.0,47646.0,49501.0,48849.0,52870.0,53535.0,56421.0,57405.0,58406.0
31,Asotin,59,22897.0,24206.0,25110.0,26251.0,27359.0,27873.0,28288.0,29813.0,31499.0,32477.0,33524.0,32590.0,35920.0,37013.0,37080.0,37140.0,37657.0,39768.0,40558.0,42945.0,40035.0,39820.0,40171.0,41703.0,42174.0,43368.0,46107.0,47020.0,51767.0,50746.0,54776.0,53377.0,56438.0,59675.0
20,Kittitas,484,20489.0,21773.0,22793.0,23828.0,25126.0,26061.0,27089.0,28626.0,29775.0,31495.0,32546.0,34206.0,36203.0,36174.0,36265.0,36640.0,37854.0,40278.0,41240.0,43582.0,41629.0,41321.0,41601.0,43098.0,46690.0,47519.0,46904.0,49275.0,55124.0,54372.0,59172.0,64727.0,62721.0,60777.0
29,Pacific,170,20029.0,20977.0,21596.0,22449.0,23372.0,24299.0,25748.0,27425.0,28974.0,30422.0,31209.0,33263.0,35123.0,34898.0,34458.0,35339.0,36420.0,38767.0,38251.0,37368.0,37898.0,36914.0,37420.0,38018.0,35934.0,40449.0,40677.0,42118.0,44418.0,45237.0,50521.0,53319.0,58715.0,60936.0
37,Columbia,16,22418.0,24478.0,24123.0,26136.0,28123.0,26779.0,29861.0,33041.0,30820.0,33060.0,33500.0,37360.0,38376.0,37328.0,37593.0,37341.0,36028.0,37211.0,41531.0,43508.0,39978.0,38474.0,38916.0,39461.0,39931.0,42886.0,39807.0,41517.0,43622.0,46550.0,56338.0,58220.0,59650.0,61115.0


In [10]:
# Extract relevant columns v1
X = df3[['2022']]  # Independent variable (income in 2022)
y = df3['Count']        # Dependent variable (count of electric vehicles)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [11]:
# Create a linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

LinearRegression()

In [12]:
y_pred = model.predict(X_test)

In [13]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 13909892.616412336


In [14]:
r_squared = r2_score(y_test, y_pred)

# Print the R-squared value
print(f'R-squared: {r_squared}')

R-squared: -10.093305752508481


In [15]:
slope = model.coef_[0]
intercept = model.intercept_
# Print regression line
print(f"Regression Line: Y = {slope:.2f} * X + {intercept:.2f}")

Regression Line: Y = 0.64 * X + -42108.43


In [26]:
df3['change'] = df3['2022'] - df3['2017']
df3.sort_values(by=['change'])

,County,Count,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,change
25,Whitman,249,21674.0,22949.0,23735.0,24389.0,25162.0,25134.0,26270.0,27838.0,28697.0,29174.0,28584.0,24596.0,24841.0,24805.0,25869.0,26752.0,26788.0,28303.0,31302.0,32604.0,32037.0,31062.0,31396.0,32570.0,43091.0,42218.0,43379.0,49946.0,52543.0,53100.0,45906.0,52066.0,53140.0,54236.0,1693.0
22,Grant,435,22372.0,24216.0,25463.0,26996.0,29172.0,29978.0,30384.0,32738.0,33977.0,35692.0,35276.0,37278.0,39556.0,40173.0,41186.0,41707.0,41824.0,44438.0,47480.0,48207.0,41195.0,42799.0,42994.0,45021.0,46036.0,50356.0,50067.0,52981.0,60344.0,58053.0,57152.0,54753.0,61815.0,65593.0,5249.0
20,Kittitas,484,20489.0,21773.0,22793.0,23828.0,25126.0,26061.0,27089.0,28626.0,29775.0,31495.0,32546.0,34206.0,36203.0,36174.0,36265.0,36640.0,37854.0,40278.0,41240.0,43582.0,41629.0,41321.0,41601.0,43098.0,46690.0,47519.0,46904.0,49275.0,55124.0,54372.0,59172.0,64727.0,62721.0,60777.0,5653.0
9,Skagit,1599,28389.0,30181.0,30979.0,31970.0,32843.0,34089.0,35278.0,36888.0,38449.0,40582.0,42381.0,42972.0,45287.0,45747.0,46747.0,48229.0,49196.0,52104.0,53874.0,54803.0,55572.0,54426.0,55085.0,56443.0,56058.0,54852.0,56322.0,55524.0,60175.0,65182.0,67551.0,78798.0,72831.0,67316.0,7141.0
13,Chelan,844,24312.0,25833.0,27592.0,28746.0,30148.0,31547.0,32164.0,33918.0,35662.0,37175.0,37316.0,39439.0,41653.0,41731.0,42918.0,43696.0,44422.0,46522.0,44964.0,44013.0,46780.0,45478.0,46275.0,47265.0,51713.0,50825.0,53068.0,55109.0,60791.0,60747.0,59838.0,61546.0,64700.0,68016.0,7225.0
14,San Juan,821,31278.0,32388.0,33298.0,34893.0,36059.0,38412.0,37596.0,39037.0,41134.0,43253.0,43491.0,44568.0,45369.0,45809.0,47688.0,51217.0,52929.0,55794.0,57026.0,56784.0,55133.0,53041.0,53916.0,55025.0,60872.0,58782.0,59260.0,61391.0,61268.0,64058.0,69113.0,74533.0,71678.0,68932.0,7664.0
31,Asotin,59,22897.0,24206.0,25110.0,26251.0,27359.0,27873.0,28288.0,29813.0,31499.0,32477.0,33524.0,32590.0,35920.0,37013.0,37080.0,37140.0,37657.0,39768.0,40558.0,42945.0,40035.0,39820.0,40171.0,41703.0,42174.0,43368.0,46107.0,47020.0,51767.0,50746.0,54776.0,53377.0,56438.0,59675.0,7908.0
36,Ferry,25,25170.0,27355.0,27236.0,27841.0,27829.0,29587.0,28910.0,30091.0,30489.0,30412.0,30388.0,31175.0,33129.0,33488.0,33182.0,33867.0,34452.0,34828.0,37001.0,38093.0,38284.0,36712.0,36921.0,37548.0,39596.0,41343.0,40340.0,42330.0,45546.0,42212.0,47463.0,47722.0,50565.0,53577.0,8031.0
12,Jefferson,848,25197.0,27294.0,28465.0,29171.0,29885.0,30606.0,31058.0,32530.0,34282.0,36404.0,37869.0,33565.0,35299.0,35723.0,36136.0,38014.0,39746.0,43099.0,44511.0,45995.0,45225.0,43814.0,44348.0,46651.0,46957.0,50161.0,52887.0,54864.0,54884.0,57485.0,60556.0,66386.0,61548.0,63641.0,8757.0
34,Adams,40,24604.0,28024.0,27301.0,28328.0,31277.0,29537.0,29604.0,31806.0,31795.0,34073.0,33888.0,35292.0,37839.0,38306.0,38934.0,39235.0,39105.0,41298.0,42299.0,42455.0,41102.0,40656.0,41068.0,42354.0,43541.0,45712.0,47646.0,49501.0,48849.0,52870.0,53535.0,56421.0,57405.0,58406.0,9557.0


In [34]:

df3['predicted_output'] = rf.predict(df3[['2022']])
df3['diff'] = df3['predicted_output'] - df3['Count']
df3.sort_values(by='diff')

,County,Count,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,change,predicted_output,diff
0,King,75383,36179.0,38633.0,39831.0,41647.0,42234.0,43810.0,45397.0,46726.0,48271.0,51266.0,53157.0,53937.0,55220.0,56098.0,56952.0,61565.0,61225.0,63745.0,65489.0,67027.0,65877.0,65383.0,66294.0,68313.0,71122.0,75045.0,80998.0,84897.0,88466.0,93707.0,102338.0,102620.0,110351.0,118664.0,30198.0,43955.333333,-31427.666667
3,Clark,8451,31800.0,33734.0,34250.0,35847.0,37418.0,38828.0,40525.0,43044.0,45705.0,47252.0,48376.0,49320.0,51610.0,50518.0,50520.0,50474.0,52108.0,56994.0,57621.0,57999.0,54370.0,54581.0,54951.0,56054.0,57852.0,61711.0,63639.0,66782.0,71922.0,71659.0,80407.0,76929.0,83790.0,87397.0,15475.0,6730.400000,-1720.600000
2,Pierce,11017,30412.0,33380.0,33976.0,35417.0,36210.0,37105.0,38222.0,39878.0,42596.0,44333.0,45204.0,42555.0,44965.0,45581.0,47084.0,49151.0,50678.0,55506.0,56426.0,57674.0,56555.0,55531.0,56114.0,57162.0,57238.0,59998.0,59566.0,61042.0,65517.0,71208.0,78779.0,80236.0,85492.0,91092.0,25575.0,9377.800000,-1639.200000
4,Thurston,5097,30976.0,33139.0,34851.0,36676.0,37766.0,38939.0,39971.0,41497.0,43748.0,45843.0,46975.0,48457.0,50885.0,51111.0,51243.0,52043.0,54914.0,57985.0,60576.0,63009.0,60978.0,60038.0,60621.0,62009.0,63408.0,65288.0,61676.0,63286.0,68765.0,69757.0,77890.0,79769.0,81544.0,83358.0,14593.0,3647.200000,-1449.800000
6,Spokane,3542,25769.0,27337.0,28355.0,29570.0,30482.0,31631.0,32256.0,33517.0,34920.0,36389.0,37308.0,39401.0,40525.0,40872.0,42533.0,44836.0,44538.0,45753.0,47848.0,48876.0,46983.0,46320.0,46846.0,48265.0,48312.0,50856.0,48189.0,49482.0,49954.0,55535.0,59976.0,60827.0,65571.0,70685.0,20731.0,2337.866667,-1204.133333
7,Whatcom,3529,28367.0,31097.0,31953.0,32599.0,32958.0,34031.0,34893.0,36492.0,37553.0,39188.0,40005.0,37044.0,39301.0,39568.0,40486.0,41151.0,43372.0,46879.0,49778.0,50443.0,49761.0,49294.0,49775.0,51268.0,50879.0,53481.0,54522.0,55710.0,60028.0,61244.0,68656.0,70463.0,71739.0,73038.0,13010.0,2709.600000,-819.400000
8,Benton,1756,32593.0,35012.0,37093.0,39228.0,41153.0,42790.0,42540.0,42817.0,44057.0,45665.0,47044.0,49389.0,52598.0,53736.0,55409.0,57613.0,56200.0,54942.0,55361.0,57114.0,58496.0,60070.0,60608.0,62739.0,63710.0,63157.0,62071.0,62282.0,63502.0,67912.0,72847.0,75882.0,77339.0,78824.0,15322.0,1050.966667,-705.033333
14,San Juan,821,31278.0,32388.0,33298.0,34893.0,36059.0,38412.0,37596.0,39037.0,41134.0,43253.0,43491.0,44568.0,45369.0,45809.0,47688.0,51217.0,52929.0,55794.0,57026.0,56784.0,55133.0,53041.0,53916.0,55025.0,60872.0,58782.0,59260.0,61391.0,61268.0,64058.0,69113.0,74533.0,71678.0,68932.0,7664.0,296.666667,-524.333333
9,Skagit,1599,28389.0,30181.0,30979.0,31970.0,32843.0,34089.0,35278.0,36888.0,38449.0,40582.0,42381.0,42972.0,45287.0,45747.0,46747.0,48229.0,49196.0,52104.0,53874.0,54803.0,55572.0,54426.0,55085.0,56443.0,56058.0,54852.0,56322.0,55524.0,60175.0,65182.0,67551.0,78798.0,72831.0,67316.0,7141.0,1245.500000,-353.500000
19,Grays Harbor,513,23042.0,24053.0,24677.0,26038.0,27002.0,27252.0,28638.0,29792.0,31368.0,33167.0,34160.0,36410.0,37440.0,38102.0,38305.0,38367.0,39943.0,42029.0,43126.0,42646.0,39927.0,39452.0,39836.0,40354.0,40323.0,41687.0,43902.0,44627.0,44080.0,45139.0,59346.0,54034.0,55018.0,56020.0,11940.0,209.066667,-303.933333


In [16]:
rf = RandomForestRegressor(n_estimators=30, random_state=52)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=30, random_state=52)

In [17]:
rfy_pred = rf.predict(X_test)

In [18]:
mse2 = mean_squared_error(y_test, rfy_pred)
print(f'Mean Squared Error (Random Forest Regression): {mse}')

Mean Squared Error (Random Forest Regression): 13909892.616412336


In [19]:
r_squared3 = r2_score(y_test, rfy_pred)

# Print the R-squared value
print(f'Tree R-squared: {r_squared3}')

Tree R-squared: 0.7824224654024496


In [21]:
# Plot the results
plt.scatter(X_test, y_test, color='black', label='Actual Data')

# regular Prediction
plt.plot(X_test, model.predict(X_test), color='blue', linewidth=3, label='SLR Model')

# tree Prediction
plt.plot(X_test, rf.predict(X_test), color='red', linewidth=3, label='Random Forest Regressor Model')

plt.xlabel('Median income in 2022 ($)')
plt.ylabel('Count of Electric Vehicles')
plt.title('EV Count vs Income (2022)')
plt.ylim(0)
plt.legend()
plt.grid(True)
plt.show()

In [1]:
df3

NameError: name 'df3' is not defined